# Validate NN on VMH unseen database
<b>Author</b>: Ian Coleman <br/>
<b>Function</b>: Let's take the NN developed in Opa/ and test it out on virtual metabolic human data

In [51]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from random import randint
import random
from sklearn.model_selection import train_test_split
import sklearn
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from pandas_ml import ConfusionMatrix
import json
import subprocess
import pickle
import math
import scipy.io
import urllib.request


#Set random seed
np.random.seed(1606)

In [52]:
# # Import VMH
# vmh = scipy.io.loadmat('data/Recon3D_301/Recon3D_301.mat')

### Import VMH Disease List

In [172]:
# Import VMH diseases list
v_dis = pd.read_csv('data/recon-store-diseases-1.tsv', sep='\t')

In [173]:
v_dis.shape

(257, 27)

In [174]:
v_dis.sample(2)

,abbreviation,name,dtype,subtype,inheritance,organ,omim,prevalence,references,ghr,...,genereviews,clingendosage,igsr1000genoms,gwascataloge,gwascentral,geno2mp,clinvar,lovd,malacard,omim_symptons
53,DBPD,D-Bifunctional Protein Deficiency,Inherited metabolic disorder,Fatty acid beta oxidation,Autosomal recessive,"Nervous system, heart, liver",261515,NaN,NaN,d-bifunctional-protein-deficiency,...,NaN,HSD17B4,ENSG00000133835,HSD17B4,HSD17B4,HSD17B4,601860.0,HSD17B4,d_bifunctional_protein_deficiency,261515.0
154,IND,Infantile Neuroaxonal Dystrophy,Inherited metabolic disorder,Lipid disorder,Autosomal recessive,"Nervous system, muscle",256600,Infantile neuroaxonal dystrophy is a very rare...,http://www.ncbi.nlm.nih.gov/bookshelf/br.fcgi?...,infantile-neuroaxonal-dystrophy,...,NBK1675,PLA2G6,ENSG00000184381,PLA2G6,PLA2G6,PLA2G6,603604.0,PLA2G6,NaN,256600.0


In [56]:
# v_dis = v_dis[['abbreviation', 'name', 'dtype', 'subtype', 'organ', 'omim', 'ghr', 'orphanet',
#  'cellLines',
#  'clinicaltrials',
#  'eurogenetest',
#  'geneticalliance',
#  'gard',
#  'igsr1000genoms',
#  'gwascentral',
#  'clinvar',
#  'malacard',
#  'omim_symptons']]

In [138]:
v_dis = v_dis[['abbreviation', 'name', 'omim', 'gwascentral']]

In [144]:
v_dis = v_dis.drop(v_dis.index[[2]])

In [151]:
v_dis['abbreviation'] = v_dis.abbreviation.astype(str)

### Get paired chems for these diseases
Using the above disease abbreviation, API call for associated chems from VMH

In [38]:
def get_input_IDs (file_path):
    """
    Input: Txt file path, one ID on each line
    Output: 
    """
    with open (file_path, 'r') as file_object:
        lines = file_object.readlines()
        lines = [i.strip() for i in lines] # strip 
        return(lines)

def create_map_ctd_cid (ctd_ids):
    """
    Input: LIST ctd_ids
    Output: DICT ctd ids as keys and cid ids as values
    """
    map_ctd_cid = {}
    for item in ctd_ids:
        url = "http://pubchem.ncbi.nlm.nih.gov/rest/pug/substance/sourceid/Comparative%20Toxicogenomics%20Database/" + item + "/cids/TXT/"
        try:
            map_ctd_cid[item] = urllib.request.urlopen(url).read().strip()
        except Exception:
            print('Exception caught: ', Exception)
            continue  # or you could use 'continue'
    return map_ctd_cid
        
def save_obj(obj, name):
    with open('./'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

if __name__ == "__main__":
    input_IDs = get_input_IDs(argv[1])
    mapper = create_map_ctd_cid(input_IDs)
    save_obj(mapper, 'ctd_cid_map')
    print(map_ctd_cid)



SyntaxError: invalid syntax (<ipython-input-38-21fb384a9406>, line 1)

In [149]:
import pdb
def get_met(abbrev):
    """
    Given a disease abbreviation from VMH make api call to get assoc metabolite
    """
    url = 'https://www.vmh.life/_api/biomarkers/?disease=' + abbrev
    response = urllib.request.urlopen(url).read().strip() # make api call 
    response = json.loads(response) # turn from byte to dict
    metName, value, cid, chebi, chembl = [np.nan] * 5
    try:
        value = response['results'][0]['value']
        metName = response['results'][0]['metabolite']['fullName']
        cid = response['results'][0]['metabolite']['pubChemId']
        chebi = response['results'][0]['metabolite']['cheBlId']
        chembl = response['results'][0]['metabolite']['chembl']
    except IndexError:
        pass
    return [metName, value, cid, chebi, chembl]

In [150]:
v_dis['stuff'] = v_dis.abbreviation.map(lambda x: get_met(x))

In [152]:
# split the new column into multiple
v_dis[['metName', 'value', 'cid', 'chebi', 'chembl']] = pd.DataFrame(v_dis.stuff.values.tolist(), index= v_dis.index)

In [164]:
v_dis.sample(7)

,abbreviation,name,omim,gwascentral,stuff,metName,value,cid,chebi,chembl
121,IVA,Isovaleric Acidemia,243500,IVD,"[Isovaleryl Carnitine, Increased, 6426851, 730...",Isovaleryl Carnitine,Increased,6426851,73025,
46,CPT2,Carnitine Palmitoyltransferase 2 Deficiency,"608836, 600649",CPT2,"[Tetradecenoylcarnitine, Increased, 53477791, ...",Tetradecenoylcarnitine,Increased,53477791,84634,
203,TYR1,Tyrosinemia Type I,276700,FAH,"[L-Tyrosine, Increased, 6057, 17895, CHEMBL925]",L-Tyrosine,Increased,6057,17895,CHEMBL925
125,TFPD,Trifunctional Protein Deficiency,609016,HADHA,"[Lauroyl Carnitine, Increased, 168381, 77086, ]",Lauroyl Carnitine,Increased,168381,77086,
155,CIT2,Type II Citrullinemia,603471,SLC25A13,"[Citrulline, Increased, 9750, 16349, CHEMBL444...",Citrulline,Increased,9750,16349,CHEMBL444814
161,OTC,Ornithine Transcarbamylase Deficiency,311250,OTC,"[Citrulline, Decreased, 9750, 16349, CHEMBL444...",Citrulline,Decreased,9750,16349,CHEMBL444814
106,HMG,HMG-Coenzyme A Lyase Deficiency,246450,HMGCL,"[3-Hydroxy-Isovaleryl Carnitine, Increased, 53...",3-Hydroxy-Isovaleryl Carnitine,Increased,53915061,73027,


In [154]:
v_dis.shape

(256, 10)

In [161]:
# Exclude those without a chemical associated
v_dis = v_dis.dropna()
print(v_dis.shape)

(34, 10)


In [167]:
# Exclude those with negative chem association - we only have a model for positive
v_dis = v_dis[v_dis.value == 'Increased']

(29, 10)

In [122]:
# # Exploring api
# url = 'https://www.vmh.life/_api/biomarkers/?disease=GD'
# response = urllib.request.urlopen(url).read().strip()
# json.loads(response)
# response['results'][0]['metabolite']['fullName']

### Get assoc genes for diseases (i) via given gene assoc in VMH


In [176]:
# Export gene name list 
genes = v_dis.gwascentral
np.savetxt(r'v_genes.txt', genes.values, fmt='%s')

### NOTE the next step is MANUAL
You need to go to https://www.uniprot.org/uploadlists/ and give it the created v_genes.txt file, ask it to convert
Gene Names to uniprot ID. Then download this as UniprotIDs.tab (as uncompressed, mapping table) to this folder

In [179]:
# Import manually generated file of geneID --> uniprotID
df_uni_ids = pd.read_csv('data/UniprotIDs.tab', sep='\t',usecols=[0,1])
df_uni_ids.columns = ['GeneID', 'UniprotID']
df_uni_ids['GeneID'] = df_uni_ids.GeneID.astype(str)

In [193]:
df_uni_ids.sample(3)

,GeneID,UniprotID,OMIM
1506,ATP6V1B1,C9JNS9,267300
837,GSS,A0A2R8YF34,231900
792,ALDOA,H3BUK5,611881


In [192]:
# Let's use this gene-uniprot df as our base, add disease into it via map
gen2dis = dict(zip(v_dis.gwascentral, v_dis.omim))
df_uni_ids['OMIM'] = df_uni_ids.GeneID.map(lambda x: gen2dis.get(x))

In [ ]:
# Now turn unprot to go func
# import goa file (uniprot ID to go_functions)
go_funcs = pd.read_csv('../goa_human.gaf', header=None, skiprows=30, sep='\t')

In [195]:
# Cut out all cols except uniprot ids and go_funcs, rename these
go_funcs = go_funcs.rename(columns={ go_funcs.columns[1]: "UniprotID" })
go_funcs = go_funcs.rename(columns={ go_funcs.columns[4]: "gofunc" })
col_list = ['UniprotID', 'gofunc']
df_go = go_funcs[col_list]

In [196]:
df_uni_ids.head()

,GeneID,UniprotID,OMIM
0,SLC12A1,A0A2R8Y6V7,601678
1,SLC12A1,H0YLJ2,601678
2,SLC12A1,H0YMG9,601678
3,SLC12A1,H0YNW0,601678
4,SLC12A1,L8E9E3,601678


In [197]:
# Merge the go functions into our existing chem-uniprotID and dis-uniprotID dfs
df_uni_ids = df_uni_ids.merge(df_go, on='UniprotID', how='outer').dropna()

In [198]:
df_uni_ids.head()

,GeneID,UniprotID,OMIM,gofunc
5,SLC12A1,Q13621,601678,GO:0005886
6,SLC12A1,Q13621,601678,GO:0006811
7,SLC12A1,Q13621,601678,GO:0008511
8,SLC12A1,Q13621,601678,GO:0016020
9,SLC12A1,Q13621,601678,GO:0016021


In [199]:
# Create a col with the full go url
df_uni_ids['go_url'] = '<' + 'http://purl.obolibrary.org/obo/' + df_uni_ids.gofunc.str.replace(':', '_')  + '>'

In [202]:
# Grab just the columns we want to output (diseaseID and go_url/ chemicalID and go_url)
col_list_d = ['OMIM', 'go_url']
df_d = df_uni_ids[col_list_d]

In [ ]:
# Output an association file for each of chem and dis
np.savetxt(r'associations_c.txt', df_c.values, fmt='%s')
np.savetxt(r'associations_d.txt', df_d.values, fmt='%s')

In [ ]:
# Merge these two into one single file
subprocess.call('cat associations_c.txt > myassociations', shell=True)
subprocess.call('cat associations_d.txt >> myassociations', shell=True)

In [ ]:
# Create entities.lst to inform opa2vec which entities we want vectors for
entities = df_d.DiseaseID.unique().tolist() + df_c.ChemicalID.unique().tolist()
np.savetxt(r'entities.lst', entities, fmt='%s')

In [ ]:
# Actually we have to make these into vectors at the same time as the opa-nn vecs so
# 1. get chem-gofuncs
# 2. integrate both into vec creation along with opa-nn vecs

### Get assoc genes for diseases (ii) pair them to CTD via semantic matching/omim

In [8]:
# # Map the vmh disease abbreviations to our disease list
# # import diseases
# c_dis = pd.read_csv('../ctd-to-nt/all-diseases-w-genes-ctd-idsnamesgenes.csv')
# print('CTD chems: ', c_dis.shape[0])

CTD chems:  30272


In [9]:
# c_dis.head()

,GeneID,DiseaseName,DiseaseID,DirectEvidence
0,50518,Diabetes Mellitus,MESH:D003920,marker/mechanism
1,50518,"Diabetes Mellitus, Type 2",MESH:D003924,marker/mechanism
2,50518,Liver Neoplasms,MESH:D008113,marker/mechanism
3,50518,Neoplasms,MESH:D009369,marker/mechanism
4,50518,Obesity,MESH:D009765,marker/mechanism


In [10]:
# # Match up diseases in two ways, OMIM and semantic similarity (may be ~ no matches...)
# # Use a measure of distance to match up disease names from ctd and from VMH 
# from difflib import SequenceMatcher
# import pdb

# def similar(a, b):
#     return SequenceMatcher(None, a, b).ratio()

# def create_map(std_list, flawed_list):
#     flawed_list = (n for n in flawed_list)
#     team_map = {}
#     best_score = {}
#     for team in flawed_list:
#         scores = [similar(team, std_team) for std_team in std_list]
#         highest = max(scores)
#         if highest > 0.8:
#             index = scores.index(max(scores))
#             team_map[team] = std_list[index]
#     return team_map

In [11]:
## KEEP commented out unless you haven't made this map yet
# mapboy = create_map(v_dis.name, c_dis.DiseaseName.unique())

# # Here we export the dictionary in a way that's easily imported as dict
# import pickle 

# with open('Uniprot_HINO_map'+ '.pkl', 'wb') as f:
#         pickle.dump(mapboy, f, pickle.HIGHEST_PROTOCOL)

In [12]:
# ## The commented section above makes a map of vmh dis to ctd dis, importing here (computationally expensive) 
# def load_obj(name):
#     with open(name + '.pkl', 'rb') as f:
#         return pickle.load(f)

# vdis2cdis = load_obj('vmhdis_to_ctddis')

In [31]:
# vdis2cdis

In [32]:
# # These are the incorrect mappings I've identified for a 0.8 similarity cutoff
# remove = ('Turner Syndrome', 'Werner Syndrome', 'Enterokinase Deficiency', 'Prolidase Deficiency')
# vdis2cdis = {key: vdis2cdis[key] for key in vdis2cdis if key not in remove}

In [33]:
# sorted(c_dis.DiseaseName.unique())[700:]

In [34]:
# sorted(v_dis.name)

In [13]:
# API script to get disease-marker pairs from VMH